In [ ]:
"""
B-tree

a balanced search tree with more than one value per node

Properties
1. Every node has n keys stored in non-decreasing order; it is
either a leaf node or an internal node;
2. Each internal node contains n+1 pointers to its children; while
leaf nodes have no children;
3. Keys of a node separate the ranges of keys in each subtree;
4. All leafs have the same height;
5. Node has a minimum degree t; Every node other than root must 
contains at least t-1 at most 2t-1 keys.

"""
class Node():
    def __init__(self):
        self._values = []
        
        
class BTree():
    def __init__(self):
        pass
    def search(self, value):
        pass
    def _split_child(self, x, i)
        pass
    def insert(self, k):
        pass
    def insert_nonfull(self, k):
        pass
    def delete(self, k):
        pass